<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/Jazykov%C3%BD_model_Transformer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Jazykový model Transformer**

https://monica.im/share/chat?shareId=CyczXVYLtNuYQpNQ

# 1. Příprava prostředí

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer

# Nastavení device na GPU pokud je dostupné, jinak CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Načtení tokenizeru pro multijazyčný BERT model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

print(f'\nUsing {device} device')

# 2. Event. instalace PyTorch

In [ ]:
# Příkaz pro instalaci PyTorch, pokud je potřeba
# !pip install torch torchvision


# 3. Příprava dat

In [ ]:
# ad kytice
!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/100x%20kytice.txt
ggg = '100x kytice.txt'

--2024-04-25 11:31:26--  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/100x%20kytice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8069800 (7.7M) [text/plain]
Saving to: ‘100x kytice.txt’

100x kytice.txt     100%[===================>]   7.70M  36.1MB/s    in 0.2s    

2024-04-25 11:31:27 (36.1 MB/s) - ‘100x kytice.txt’ saved [8069800/8069800]



In [ ]:
# ad EU
# !wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/EU/eu!!!.txt
# ggg = 'eu!!!.txt'

In [ ]:
# Funkce pro tokenizaci a přípravu vstupů a segmentů
def tokenize_and_prepare_inputs_segments(file_path, max_length=512, segment_length=510):
    inputs = {'input_ids': [], 'attention_mask': []}
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = text.split()
        segments = [' '.join(words[i:i+segment_length]) for i in range(0, len(words), segment_length)]

        for segment in segments:
            segment_inputs = tokenizer(segment, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length)
            inputs['input_ids'].append(segment_inputs['input_ids'])
            inputs['attention_mask'].append(segment_inputs['attention_mask'])

    inputs['input_ids'] = torch.cat(inputs['input_ids'], dim=0)
    inputs['attention_mask'] = torch.cat(inputs['attention_mask'], dim=0)

    return inputs

# Příklad tokenizace a přípravy vstupů pro český text s rozdělením na segmenty
file_path = ggg  # Zde je potřeba upravit cestu k vašemu souboru
inputs_segments = tokenize_and_prepare_inputs_segments(file_path)

print(f"Rozměry input_ids: {inputs_segments['input_ids'].shape}")
print(f"Rozměry attention_mask: {inputs_segments['attention_mask'].shape}")

"""
# Kontrola rozměrů tensorů
batch_size = 32
assert inputs_segments['input_ids'].shape[0] % batch_size == 0, "Velikost batche neodpovídá počtu segmentů"
"""
input_ids = inputs_segments['input_ids'].to(device)
attention_mask = inputs_segments['attention_mask'].to(device)


# Vytvoření tensoru targets posunutím input_ids o 1 doprava
# Poslední token každé sekvence je odstraněn a na začátek je přidán token, který může být například [PAD] token nebo jiný speciální token
# Zde používáme 0 jako placeholder pro [PAD] token nebo jiný speciální token, který indikuje začátek sekvence
targets = torch.cat((input_ids[:, 1:], input_ids[:, :1]*0), dim=1).to(device)

# Vytvoření datasetu obsahujícího input_ids, attention_mask (pokud ho máte) a targets
# Předpokládáme, že máte tensor attention_mask z předchozího kroku
# attention_mask = ...
dataset = TensorDataset(input_ids, attention_mask, targets)

# Vytvoření DataLoaderu z datasetu
batch_size = 32  # Nastavte podle vašich potřeb
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)







# Zde můžete pokračovat s definicí modelu a trénováním



In [ ]:
#  ???????????????? validation

from torch.utils.data import TensorDataset, DataLoader

# Předpokládáme, že máte validační input_ids a attention_mask, zatím se rovnají trénovacím datům....
# Tady byste měli mít kód pro přípravu validačních input_ids a attention_mask
valid_input_ids = inputs_segments['input_ids']
valid_attention_mask = inputs_segments['attention_mask']

# Vytvoření targets posunutím validačních input_ids o 1 doprava
valid_targets = torch.cat((valid_input_ids[:, 1:], valid_input_ids[:, :1]), dim=1)

# Vytvoření validačního datasetu
valid_dataset = TensorDataset(valid_input_ids, valid_attention_mask, valid_targets)

# Vytvoření validačního DataLoaderu
valid_batch_size = 32  # Můžete nastavit podle vašich potřeb a možností hardwaru
valid_dataloader = DataLoader(valid_dataset, batch_size=valid_batch_size, shuffle=False)  # Pro evaluaci obvykle nepotřebujeme míchat data


In [ ]:
"""
def create_mask(input_ids):
    # Vytvoření masky pro padding tokeny (předpokládáme, že padding token má ID 0)
    return (input_ids != 0).long()
"""

# 4. Vytvoření modelu

In [ ]:
import math
import torch
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

# Nastavení parametrů modelu
ntokens = 20000  # velikost slovníku - podle vašeho datasetu - zatím OK , lze i zvětšit, ale tak prodloužit výpočet...
emsize = 256  # velikost embeddingů
nhid = 256  # velikost skryté vrstvy
nlayers = 2  # počet Transformer bloků
nhead = 4  # počet hlav v multi-head attention mechanismu
dropout = 0.2  # dropout

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)


In [ ]:

# Přesunutí tokenizovaných vstupů na správné zařízení
input_ids = inputs_segments['input_ids'].to(device)
attention_mask = inputs_segments['attention_mask'].to(device)


# 5. Trénování modelu

In [ ]:
# Předpokládáme, že input_ids a attention_mask jsou již připraveny a na správném zařízení

# Definice loss funkce a optimalizátoru
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Trénovací cyklus
num_epochs = 5  # Nastavte počet epoch podle vašich potřeb



In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.
    for batch in dataloader:
        b_input_ids, b_attention_mask, b_targets = batch

        b_input_ids = b_input_ids.to(device)
        max_index = 19999  # Maximální index pro embedding vrstvu
        unk_index = 1  # Předpokládáme, že index 1 reprezentuje [UNK] token
        # Nahrazení všech indexů větších než max_index indexem pro [UNK] token
        b_input_ids = torch.where(b_input_ids > max_index, unk_index, b_input_ids).to(device)

        b_attention_mask = b_attention_mask.to(device)
        b_targets = b_targets.to(device)

        optimizer.zero_grad()

        # Generování src_mask pro aktuální batch
        seq_length = b_input_ids.size(1)  # Délka sekvence pro aktuální batch
        src_mask = model.generate_square_subsequent_mask(seq_length).to(device)
        src_mask = src_mask.unsqueeze(0)  # Přidáme rozměr pro batch_size, pokud je potřeba

        # Nyní máme src_mask s rozměrem (1, seq_length, seq_length)
        # Můžete potřebovat upravit tento kód, aby odpovídal očekávaným rozměrům vaší implementace
        output = model(b_input_ids, src_mask)

        # Výpočet loss
        loss = criterion(output.view(-1, ntokens), b_targets.view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Výpis průměrného loss po každé epochě
    print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {total_loss / len(dataloader)}")



"""
    # Tento kód by měl být spuštěn po trénovacím cyklu pro každou epochu nebo po dokončení všech epoch
    model.eval()  # Přepnutí modelu do evaluačního režimu
    valid_loss = 0.
    with torch.no_grad():  # Vypnutí výpočtu gradientů pro evaluaci
        for batch in valid_dataloader:
            b_input_ids, b_attention_mask, b_targets = batch

            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_targets = b_targets.to(device)

            output = model(b_input_ids, b_attention_mask)
            loss = criterion(output.view(-1, ntokens), b_targets.view(-1))

            valid_loss += loss.item()

        print(f"Validation Loss: {valid_loss / len(valid_dataloader)}")

"""



In [ ]:
print("Max index in b_input_ids:", b_input_ids.max().item())
print(ntokens)


In [ ]:
print("Number of tokens in embedding layer:", model.encoder.weight.size(0))


# 6. Evaluace a použití modelu

In [ ]:
# Zde byste měli přidat kód pro evaluaci modelu a generování textu.



# 7. Ukládání a načítání modelu

In [ ]:
# Uložení modelu do Google Drive
torch.save(model.state_dict(), '/content/drive/My Drive/model_state_dict.pth')

# Načtení modelu
model.load_state_dict(torch.load('/content/drive/My Drive/model_state_dict.pth'))
